In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import brown
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer, PorterStemmer
#from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB

In [2]:
df=pd.read_csv("Train.csv")

In [3]:
df.head()

,review,label
0,mature intelligent and highly charged melodram...,pos
1,http://video.google.com/videoplay?docid=211772...,pos
2,Title: Opera (1987) Director: Dario Argento Ca...,pos
3,I think a lot of people just wrote this off as...,pos
4,This is a story of two dogs and a cat looking ...,pos


In [4]:
review=df['review'].values

In [5]:
sw=set(stopwords.words('english'))
def remove_stopwords(text,stopword):
    useful=[i for i in text if i not in stopword]
    return useful

In [6]:
tokenizer=RegexpTokenizer('[a-zA-z]+')
def myTokenizer(document):
    words=tokenizer.tokenize(document.lower())
    #Remove stopwords
    words=remove_stopwords(words,sw)
    return words

In [7]:
ps=PorterStemmer()
wn=WordNetLemmatizer()
for count,i in enumerate(df['review']):
    #if count==1000:   break
    words=myTokenizer(i)
    #print(words)
    words=[ps.stem(j) for j in words]
        
    
    df.loc[count,'review']=' '.join(words)
    
    

In [8]:
df.head(10)

,review,label
0,matur intellig highli charg melodrama unbelive...,pos
1,http video googl com videoplay docid hl en dis...,pos
2,titl opera director dario argento cast cristin...,pos
3,think lot peopl wrote anoth one tom cruis weir...,pos
4,stori two dog cat look way back home old wise ...,pos
5,steve carel come first star role year old virg...,pos
6,go write requir howev summari put top way word...,neg
7,ok riski move rent flick thought noth lose wel...,neg
8,cannib pair cinemat refer delicatessen plot st...,pos
9,one great modern kung fu film lot review seem ...,pos


In [9]:
review=df['review'].values

In [10]:
for count,i in enumerate(df['label']):
    if i=='pos':
        df.loc[count,'label']=1
    else:
        df.loc[count,'label']=0

In [11]:
df.head(10)

,review,label
0,matur intellig highli charg melodrama unbelive...,1
1,http video googl com videoplay docid hl en dis...,1
2,titl opera director dario argento cast cristin...,1
3,think lot peopl wrote anoth one tom cruis weir...,1
4,stori two dog cat look way back home old wise ...,1
5,steve carel come first star role year old virg...,1
6,go write requir howev summari put top way word...,0
7,ok riski move rent flick thought noth lose wel...,0
8,cannib pair cinemat refer delicatessen plot st...,1
9,one great modern kung fu film lot review seem ...,1


# Buliding Vocab

In [12]:
#cv=TfidfVectorizer()
#cv=CountVectorizer(ngram_range=(1,3))
cv=CountVectorizer(ngram_range=(1,1))
#tf=TfidfVectorizer()
vectorized_corpus=cv.fit_transform(review[:35000])

In [13]:
len(cv.vocabulary_)

59273

In [14]:
X_train=vectorized_corpus

In [15]:
Y_train=df.loc[:34999,'label'].values
#Y_cross=df.loc[8000:15999,'label'].values.astype('int')
#print(cv.vocabulary_)
(Y_train.shape)
(Y_train[0])

1

In [16]:
Y_train=Y_train.astype('uint8')
print(X_train.shape,Y_train.shape)
X_train[0]

(35000, 59273) (35000,)


<1x59273 sparse matrix of type '<class 'numpy.int64'>'
	with 19 stored elements in Compressed Sparse Row format>

# Naive Bayes Classifier

In [ ]:
#gnb=GaussianNB()
mnb=MultinomialNB()

In [ ]:
mnb.fit(X_train,Y_train)

In [ ]:
#print(mnb.score(X_train,Y_train))

In [ ]:
#check_corpus=cv.transform(review[8000:16000])

#X_cross=check_corpus.toarray()

#X_cross.shape

#print(mnb.score(X_cross,Y_cross))

# Testing Data

In [17]:
Test=pd.read_csv("Test.csv")

In [18]:
for count,i in enumerate(Test['review']):
    #if count==1000:   break
    words=myTokenizer(i)
    #print(words)
    words=[ps.stem(j) for j in words]
        
    
    Test.loc[count,'review']=' '.join(words)

In [19]:
Test_corpus=cv.transform(Test['review'].values)
#mnb.predict()

In [20]:
#X_test=Test_corpus.toarray()

In [21]:
Test_corpus.shape

(10000, 59273)

In [ ]:
Y_test=mnb.predict(Test_corpus)

In [ ]:
sub_file=pd.DataFrame(data=Y_test,columns=['label'])

In [ ]:
for count,i in enumerate(sub_file['label']):
    if i==1:
        sub_file.loc[count,'label']='pos'
    else:
        sub_file.loc[count,'label']='neg'

In [ ]:
sub_file.head()

In [ ]:
sub_file.to_csv("submit_5_all_mnb.csv")

In [ ]:
X_train.shape

In [22]:
from keras import models
from keras.layers import Dense

In [23]:
model=models.Sequential()

In [24]:
model.add(Dense(16,activation="relu",input_shape=(59273,)))
model.add(Dense(16,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

In [25]:
model.compile(optimizer='rmsprop',
    loss="binary_crossentropy",
    metrics="accuracy")

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                948384    
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 948,673
Trainable params: 948,673
Non-trainable params: 0
_________________________________________________________________


In [27]:
X_train=X_train.todense()
Y_train=Y_train.todense()
print(X_train.shape,Y_train.shape)

AttributeError: 'numpy.ndarray' object has no attribute 'todense'

In [28]:
#import tensorflow as tf
#import scripy
X_val=(X_train[:5000,:])
Y_val=(Y_train[:5000])
X_train_new=X_train[5000:,:]
Y_train_new=Y_train[5000:]

In [29]:
print(X_train_new.shape,Y_train_new.shape,X_val.shape,Y_val.shape)

(30000, 59273) (30000,) (5000, 59273) (5000,)


In [49]:
hist=model.fit(X_train_new,Y_train_new,epochs=1,batch_size=128,validation_data=(X_val,Y_val))

235/235 [==============================] - 104s 442ms/step - loss: 0.1217 - accuracy: 0.9558 - val_loss: 0.3913 - val_accuracy: 0.8730


In [31]:
X_test=Test_corpus.todense()

In [41]:
Y_test=model.predict(X_test)

In [42]:
Y_test

array([[1.9259632e-02],
       [2.0612091e-02],
       [2.8848648e-04],
       ...,
       [9.9976492e-01],
       [9.8419440e-01],
       [9.2252791e-03]], dtype=float32)

In [43]:
ans=[]
for i in Y_test:
    if i>=0.5:
        ans.append(1)
    else:
        ans.append(0)

In [44]:
sub_file=pd.DataFrame(data=ans,columns=['label'])

In [45]:
for count,i in enumerate(sub_file['label']):
    if i==1:
        sub_file.loc[count,'label']='pos'
    else:
        sub_file.loc[count,'label']='neg'

In [46]:
sub_file.head()

,label
0,neg
1,neg
2,neg
3,pos
4,pos


In [47]:
sub_file.to_csv("submit_3_by_neural.csv")